<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 12: Use Federated Learning on MNIST
Your job is to train on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) using Federated Learning. However, the gradient shouldn't come up to central server in raw form.

In [0]:
pip install syft

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
import syft as sy
hook = sy.TorchHook(torch)

smith = sy.VirtualWorker(hook, id = "smith") #virtual worker, holds data
sally = sy.VirtualWorker(hook, id = "sally") #virtual worker, holds data

W0729 04:09:58.986621 140427676141440 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# data distribution across VirtualWorker (using federate method)
federated_train_loader = sy.FederatedDataLoader( 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((smith, sally)), 
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

In [17]:
len(test_loader)

157

##Building the network

In [0]:
# Training model parameters
class Arguments():
    def __init__(self):
        self.batch_size = 64 # training batch size 
        self.test_batch_size = 1000 # testing bacth size 
        self.epochs = 12 # epoch number
        self.seed = 1
        self.lr = 0.02  # learning rate
        self.momentum = 0.5 
        self.no_cuda = False  
        self.save_model = False
        self.log_interval = 12

args = Arguments()

torch.manual_seed(args.seed) # random number generator
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") # switching between CPU and cuda

In [0]:
torch.set_default_tensor_type(torch.cuda.FloatTensor) 

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
     
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

##Train the network

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
      
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() 
 
        if batch_idx % args.log_interval == 0:
            loss = loss.get() 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, 
                100. * batch_idx / len(train_loader), loss.item()))

##Test the network

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(1, keepdim=True) 
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [22]:
model = Classifier().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) 

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.324064
Train Epoch: 1 [768/60032 (1%)]	Loss: 2.238193
Train Epoch: 1 [1536/60032 (3%)]	Loss: 2.162025
Train Epoch: 1 [2304/60032 (4%)]	Loss: 2.024521
Train Epoch: 1 [3072/60032 (5%)]	Loss: 1.656873
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.286806
Train Epoch: 1 [4608/60032 (8%)]	Loss: 0.874594
Train Epoch: 1 [5376/60032 (9%)]	Loss: 0.881216
Train Epoch: 1 [6144/60032 (10%)]	Loss: 0.516702
Train Epoch: 1 [6912/60032 (12%)]	Loss: 0.499306
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.495695
Train Epoch: 1 [8448/60032 (14%)]	Loss: 0.445575
Train Epoch: 1 [9216/60032 (15%)]	Loss: 0.339261
Train Epoch: 1 [9984/60032 (17%)]	Loss: 0.445339
Train Epoch: 1 [10752/60032 (18%)]	Loss: 0.489623
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.216479
Train Epoch: 1 [12288/60032 (20%)]	Loss: 0.275770
Train Epoch: 1 [13056/60032 (22%)]	Loss: 0.178101
Train Epoch: 1 [13824/60032 (23%)]	Loss: 0.223266
Train Epoch: 1 [14592/60032 (24%)]	Loss: 0.270243
Train Epoch: 1 [15360/6003